In [4]:
import pandas as pd
import ast
import numpy as np
import json

In [5]:
import os
print(os.getcwd())


c:\Users\Pc - Jujuy\Desktop\proyercto individual23\PI_ML_OPS


In [7]:
# Leer el archivo Excel 
file_path = "movies_dataset.csv"
df_raw = pd.read_csv(file_path)

C:\Users\Pc - Jujuy\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
print(df_raw['spoken_languages'].head())

0             [{'iso_639_1': 'en', 'name': 'English'}]
1    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2             [{'iso_639_1': 'en', 'name': 'English'}]
3             [{'iso_639_1': 'en', 'name': 'English'}]
4             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, dtype: object


In [9]:
df_raw.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [11]:
# define columns with data of dict type to process
cols = ['spoken_languages']

In [12]:
s = df_raw[cols[0]].copy()
s.shape

(45466,)

In [13]:
s[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

In [14]:
# evaluate as a list
l = ast.literal_eval(s[0])
l

[{'iso_639_1': 'en', 'name': 'English'}]

In [15]:
# copy one column to a pandas series
s = df_raw[cols[0]].copy()
# fillna with [None]
s.fillna('[{}]', inplace=True)

l = []  # init an empty list

for i in s:
    if i == [{}]:
        # append [{}] to the list
        l += i
    else:
        # evaluate as a list
        l += ast.literal_eval(i)

In [16]:
len(l)

53306

In [17]:
l[:10]

[{'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'fr', 'name': 'Français'},
 {'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'es', 'name': 'Español'},
 {'iso_639_1': 'fr', 'name': 'Français'},
 {'iso_639_1': 'en', 'name': 'English'}]

In [18]:
def to_list_of_dict(series):
    """
    Evaluate a pandas series as a list of dict

    Input:
    "[{'one': 1, 'two': 2, 'three': 3}]"

    Output:
    [{'one': 1,
      'two': 2,
      'three' : 3}]
    """
    l = []  # init an empty list
    s = series.fillna('[{}]')  # map nan to [{}] for further eval

    # loop through the whole series
    for i in s:
        if i == [{}]:
            # append [{}] to the list
            l += i
        else:
            # evaluate as a list
            l += ast.literal_eval(i)

    return l

In [19]:
def column_conversion(col, df):
    """
    Merge a pandas series with data like list of dict back to the dataframe

    Input:
    "[{'one': 1, 'two': 2, 'three': 3}]"

    Output:
    A dataframe with the original column removed, each dict's key in a new column
    """
    l = to_list_of_dict(df[col])  # convert to list of dict
    df_right = pd.DataFrame.from_dict(l)  # convert to df
    df_merged = df.merge(df_right.add_prefix(col+'_'),  # add the original column name as prefix
                         left_index=True, right_index=True)  # merge df with df_right
    df_merged.drop(col, axis=1, inplace=True) # drop the original column

    return df_merged

In [20]:
# Test
column_conversion(cols[0], df_raw)[:3]


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,status,tagline,title,video,vote_average,vote_count,spoken_languages_iso_639_1,spoken_languages_name
0,False,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,en,English
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,en,English
2,False,Grumpy Old Men Collection,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,fr,Français


In [21]:
df_raw.to_csv(file_path, index=False)
